In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connection à la db, crée le fichier s'il n'existe pas déjà
conn = sqlite3.connect('db_foot.sqlite3') 
c = conn.cursor()

In [3]:
# Requètes de creation des différentes tables
championships = '''CREATE TABLE IF NOT EXISTS championships (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR,
    country VARCHAR,
    start_year INTEGER,
    end_year INTEGER,
    created_at TIMESTAMP,
    updated_at TIMESTAMP)'''
teams = '''CREATE TABLE IF NOT EXISTS teams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    championship_id INTEGER,
    name VARCHARD,
    city VARCHAR,
    coach_name VARCHAR,
    rank INTEGER,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY(championship_id) REFERENCES championships(id))'''
players = '''CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR,
    birthdate DATE,
    team_id INTEGER,
    position VARCHAR,
    nationality VARCHAR,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY(team_id) REFERENCES teams(id))'''
goals = '''CREATE TABLE IF NOT EXISTS goals (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER,
    player_id INTEGER,
    goal_type VARCHAR,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY (player_id) REFERENCES players(id),
    FOREIGN KEY (match_id) REFERENCES matchs(id))'''
matchs = '''CREATE TABLE IF NOT EXISTS matchs(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATIME,
    place VARCHAR,
    rainfall FLOAT,
    temperature FLOAT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP)'''
teams_matches = '''CREATE TABLE IF NOT EXISTS teams_matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER,
    team_id INTEGER,
    home BOOLEAN,
    team_goals INTEGER,
    points INTEGER,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY (team_id) REFERENCES teams(id)
    FOREIGN KEY (match_id) REFERENCES matchs(id))'''


In [4]:
# Fonctions de création et de suppression de table
def create_table(query):
    '''Create table from query specified'''
    c.execute(query)
    conn.commit()

def drop_table(table_name):
    '''Drop table specified'''
    c.execute('''DROP TABLE %s''' % (table_name,))
    conn.commit()

In [5]:
# Fonction d'insertion 
def add_championship(name, country, start_year, end_year):
    '''Add championship in table championships with values specified'''
    c.execute('''INSERT INTO championships (name, country, start_year, end_year, created_at, updated_at) VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)''', (name,country,start_year,end_year))
    conn.commit()

In [6]:
# Version pandas
def insert_table(my_df, my_table):
  '''Insert Dataframe into table specified'''
  my_df.to_sql(my_table, conn, if_exists='append', index=False)

def read_db(my_query):
  '''Read sql query, read the db an return response in a dataframe'''
  df = pd.read_sql(my_query, conn)
  return df

In [7]:
#Sandbox
#drop_table('players')
# create_table(championships)
# create_table(matchs)
# create_table(teams)
#create_table(players)
# create_table(goals)
# create_table(teams_matches)

#add_championship('Ligue 1', 'France', 2020, 2021)

In [8]:
# Lis le contenu de la table
df_championships = read_db('''SELECT * FROM championships''')
df_championships

,id,name,country,start_year,end_year,created_at,updated_at
0,1,Ligue 1,France,2020,2021,2021-06-28 12:56:07,2021-06-28 12:56:07


In [9]:
# Recup id et nom dans un dict
championships_dict = {}
for championship in range(len(df_championships)):
    championship_dict = {df_championships.iloc[championship,1] : df_championships.iloc[championship, 0]}
    championships_dict.update(championship_dict)
championships_dict

# map colonne avec dictionnaire des ids
# df_teams['championship_id'] = df_teams['championship'].map(championships_dict)
# df_teams

{'Ligue 1': 1}

In [14]:
df_matchs = pd.read_csv('match_table.csv')
df_matchs.drop(['Unnamed: 0'], axis=1, inplace=True)

In [15]:
df_matchs

,date,place,rainfall,temperature,created_at,updated_at
0,2020-08-21 19:00:00+02:00,Bordeaux,5.000000,24.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
1,2020-08-22 17:00:00+02:00,Dijon,19.000000,26.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
2,2020-08-22 21:00:00+02:00,Lille,1.000000,21.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
3,2020-08-23 13:00:00+02:00,Monaco,1.000000,27.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
4,2020-08-23 15:00:00+02:00,Lorient,1.000000,19.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
...,...,...,...,...,...,...
375,2021-05-23 21:00:00+02:00,Rennes,2.000000,15.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
376,2021-05-23 21:00:00+02:00,Nantes,1.000000,16.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
377,2021-05-23 21:00:00+02:00,Saint-Étienne,4.277397,13.279778,2021-06-30 12:01:37,2021-06-30 12:06:46
378,2021-05-23 21:00:00+02:00,Reims,1.000000,14.000000,2021-06-30 12:01:37,2021-06-30 12:06:46


In [16]:
insert_table(df_matchs, 'matchs')

read_db('''SELECT * FROM matchs''')

,id,date,place,rainfall,temperature,created_at,updated_at
0,1,2020-08-21 19:00:00+02:00,Bordeaux,5.000000,24.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
1,2,2020-08-22 17:00:00+02:00,Dijon,19.000000,26.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
2,3,2020-08-22 21:00:00+02:00,Lille,1.000000,21.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
3,4,2020-08-23 13:00:00+02:00,Monaco,1.000000,27.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
4,5,2020-08-23 15:00:00+02:00,Lorient,1.000000,19.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
...,...,...,...,...,...,...,...
375,376,2021-05-23 21:00:00+02:00,Rennes,2.000000,15.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
376,377,2021-05-23 21:00:00+02:00,Nantes,1.000000,16.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
377,378,2021-05-23 21:00:00+02:00,Saint-Étienne,4.277397,13.279778,2021-06-30 12:01:37,2021-06-30 12:06:46
378,379,2021-05-23 21:00:00+02:00,Reims,1.000000,14.000000,2021-06-30 12:01:37,2021-06-30 12:06:46


In [19]:
from scrapp_teams import df_teams

ImportError: cannot import name 'df_teams' from 'scrapp_teams' (/home/juliecharrier/Documents/Data IA/Brief_foot/brief-foot/scrapp_teams.py)